In [ ]:
# Import necessary libraries
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np, pandas as pd, matplotlib.pyplot as plt

from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Define the Autoencoder
encoding_dim = 64
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(encoding_dim, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the Autoencoder
history = autoencoder.fit(x_train, x_train,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Plot accuracy and loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Autoencoder Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

decoder=Model(encoded,decoded)

encoded_img = encoder.predict(x_test)
decoded_img = decoder.predict(encoded_img)
plt.figure(figsize=(20, 4))
for i in range(5):
    # Display original
    ax = plt.subplot(2, 5, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # Display reconstruction
    ax = plt.subplot(2, 5, i + 1 + 5)
    plt.imshow(decoded_img[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Encode the data using trained Autoencoder
encoded_train = encoder.predict(x_train)
encoded_test = encoder.predict(x_test)

# Apply K-NN on encoded data
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(encoded_train, y_train)

# Predict labels for test data
y_pred = knn.predict(encoded_test)

# Evaluate the K-NN model
acc = accuracy_score(y_test, y_pred)
print(f"K-NN accuracy on encoded data: {acc * 100:.2f}%")

from sklearn.metrics import classification_report
# Classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step 
(60000, 784)
(10000, 784)
Epoch 1/30
235/235 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 0.3253 - val_loss: 0.1413
Epoch 2/30
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - loss: 0.1331 - val_loss: 0.1132
Epoch 3/30
235/235 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 0.1105 - val_loss: 0.1013
Epoch 4/30
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - loss: 0.1011 - val_loss: 0.0954
Epoch 5/30
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 0.0957 - val_loss: 0.0915
Epoch 6/30
 28/235 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0929